<a href="https://colab.research.google.com/github/Katab-sawan/portfolio/blob/main/Options_Pricing_model.V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

***Black scholes Options Pricing Model for European options***

In finance the black scholes model is one of the most used methods to price options to which it calculates the theortical values of on option based on five key variables:



*   Underlying Price(S)
*   Strike Price(K)
*   Time to Expiration(T)
*   Risk Free Rate(r)***italicised text***
*   Volatility(sigma)







In [ ]:
from enum  import Enum
from abc import ABC, abstractmethod

class OPTION_TYPE(Enum):
    CALL_OPTION = 'Call Option'
    PUT_OPTION = 'Put Option'

class OptionPricingModel(ABC):
    #Abstract class defining interface for option pricing models.

    def calculate_option_price(self, option_type):
        #Calculates call/put option price according to the specified parameter.
        if option_type == OPTION_TYPE.CALL_OPTION.value:
            return self._calculate_call_option_price()
        elif option_type == OPTION_TYPE.PUT_OPTION.value:
            return self._calculate_put_option_price()
        else:
            return -1

    @classmethod
    @abstractmethod
    def _calculate_call_option_price(cls):
        """Calculates option price for call option."""
        pass

    @classmethod
    @abstractmethod
    def _calculate_put_option_price(cls):
        """Calculates option price for put option."""
        pass

In [ ]:
from scipy.stats import norm
import math

class BlackScholesModel(OptionPricingModel):
    def __init__(self, S, K, T, r, sigma):
        self.S = S
        self.K = K
        self.T = T
        self.r = r
        self.sigma = sigma

    def _calculate_call_option_price(self):
        d1 = (math.log(self.S / self.K) + (self.r + 0.5 * self.sigma ** 2) * self.T) / (self.sigma * math.sqrt(self.T))
        d2 = d1 - self.sigma * math.sqrt(self.T)
        return self.S * norm.cdf(d1) - self.K * math.exp(-self.r * self.T) * norm.cdf(d2)

    def _calculate_put_option_price(self):
        d1 = (math.log(self.S / self.K) + (self.r + 0.5 * self.sigma ** 2) * self.T) / (self.sigma * math.sqrt(self.T))
        d2 = d1 - self.sigma * math.sqrt(self.T)
        return self.K * math.exp(-self.r * self.T) * norm.cdf(-d2) - self.S * norm.cdf(-d1)


In [ ]:
model = BlackScholesModel(S=42, K=40, T=0.5, r=0.1, sigma=0.2)
call = model.calculate_option_price(OPTION_TYPE.CALL_OPTION.value)
put = model.calculate_option_price(OPTION_TYPE.PUT_OPTION.value)

print(f"Call Price: {call:.4f}")
print(f"Put Price:  {put:.4f}")


***Binomial Option Pricing Model For american options***

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse import csc_matrix
from scipy.stats import bernoulli

In [ ]:
S0 = 100
t = 1
u = 1.05
d = 1/u
p = 0.6

S_u = u * S0
S_d = d * S0
K = 102.5
r = 0.01

C_u = S_d - K
C_d = 0

print(S_u, S_d)

C = np.exp(-r * t) * (p*C_u + (1-p)*C_d)
print(C)

In [ ]:
N = 5

t = 1
t = t / (N-1)

S0 = 100
r = 0.01

sigma = 0.04
u = np.exp(sigma * np.sqrt(t))
u = 1/d
p = (np.exp(r*t) - d) / (u - d)

stock_prices = csc_matrix((N,N))
call_prices = csc_matrix((N,N))

stock_prices[0,0] = S0

for i in range(1,N):
  M = i + 1
  stock_prices[i,0] = d * stock_prices[i-1,0]
  for j in range(1,M):
    stock_prices[i,j] = u * stock_prices[i-1,j-1]

expiration = stock_prices[:,-1].toarray() - K
expiration_shape = (expiration.size, )
expiration = np.where(expiration >= 0, expiration, 0)

call_prices[-1,:] = expiration

for i in range(N - 2, -1, -1):
  for j in range(i+1):
    call_prices[i,j] = np.exp(-r * t) * ((1-p) * call_prices[i+1,j] + p * call_prices[i+1,j+1])


plt.spy(call_prices)
print(call_prices[0,0])